In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score
from tqdm import tqdm
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC
from xgboost import XGBClassifier
import pickle
import tensorflow as tf
import math
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from matplotlib import pyplot as plt
from sklearn.metrics import silhouette_samples
import matplotlib.cm as cm
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.decomposition import TruncatedSVD

In [2]:
df = pd.read_csv("../Data_Scrape/Dataset/updated_dataset.csv")

In [3]:
df.head()

,sp_song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,lda_topic_11,lda_topic_12,lda_topic_13,lda_topic_14,lda_topic_15,lda_topic_16,lda_topic_17,lda_topic_18,lda_topic_19,Playlists
0,6edQfeOlqbGteYixpJl3Sm,0.857,0.602,10,-8.311,1,0.0290,0.489000,0.02440,0.3170,...,0.0,0.022993,0.000000,0.122078,0.0,0.000000,0.000000,0.081108,0.386042,[165]
1,5Oe7wHPL4hdEXeF4AOayCi,0.322,0.990,8,-1.785,1,0.1710,0.000044,0.41700,0.0366,...,0.0,0.000000,0.163142,0.022999,0.0,0.144614,0.187207,0.000000,0.000000,[79]
2,6i1uWZYWabNHq2wQnoca58,0.666,0.884,9,-5.243,0,0.0429,0.506000,0.00612,0.0408,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.582131,[15]
3,4BzBtS6PBreni5hNPo2hos,0.609,0.578,9,-7.081,1,0.0414,0.296000,0.00000,0.1500,...,0.0,0.000000,0.511716,0.084905,0.0,0.000000,0.000000,0.115004,0.108140,[168]
4,0GvhHQbWSnGltjl0je61dI,0.699,0.514,4,-12.610,1,0.0315,0.587000,0.02900,0.2200,...,0.0,0.000000,0.000000,0.401954,0.0,0.000000,0.000000,0.000000,0.000000,"[30, 133]"


In [4]:
dataset = []
data = np.array(df)
for i in df.index:
    l = eval(df["Playlists"][i])
#     if len(l) > 1:
#         continue
    data[i][-1] = l
    dataset.append(data[i])
dataset = np.array(dataset)
        

In [5]:
print(len(df))
print(len(dataset))

11159
11159


In [6]:
# counts = []
# for i in range(169):
#     counts.append(0)
# for i in range(len(dataset)):
#     counts[int(dataset[i][-1])] += 1
# c = 0
# playlists_to_keep = set()
# for i in range(len(counts)):
#     if counts[i] >= 120:
#         c += 1
#         playlists_to_keep.add(str(i))        
# print(c)

In [7]:
# temp_dataset = []
# for c in dataset:
#     if str(c[-1]) in playlists_to_keep:
#         temp_dataset.append(c)
# dataset = temp_dataset
# dataset = np.array(dataset)
        

In [8]:
X = dataset[:, 1:-1]
y = dataset[:, -1]

In [9]:
standardScalar = StandardScaler()
X = standardScalar.fit_transform(X)

In [10]:
# pca = PCA(n_components = 15)
# X = pca.fit_tranform(X)
# pca.explained_variance_

In [11]:
svd = TruncatedSVD(n_components = 15, random_state = 0)
X = svd.fit_transform(X)
# pca.explained_variance_

In [12]:
print(X.shape)
print(y.shape)
print(X[0])

(11159, 15)
(11159,)
[-1.81088185 -1.63204109 -0.73234896 -0.01666838 -0.70166087  1.39663556
 -1.31109854  0.13860771 -0.70312456 -0.27157586 -0.19136596 -0.16054815
 -1.19500578  0.07239146 -0.22634698]


In [13]:
# for i in range(2, 11):
#     kmeans = KMeans(n_clusters=i, random_state=0).fit(X)
#     print(i, silhouette_score(X, kmeans.labels_, metric='euclidean')) 
#     pred = kmeans.fit_predict(X)
      
#     xx=TSNE(n_components = 2 )
#     X_TSNE=xx.fit_transform(X)
#     tsne_df = pd.DataFrame(columns = ['tsvd1','tsvd2','Ytrain'])
#     tsne_df["tsvd1"]=X_TSNE[:,0]
#     tsne_df["tsvd2"]=X_TSNE[:,1]
#     tsne_df["Ytrain"]=pred
#     # creating a dataframe of both dimensions of TSNE model and Ytrain to visualize plots

#     # plotting our TSNE plot with labels on side
#     plt.figure(figsize=(16,10))
#     sns.scatterplot(x="tsvd1", y="tsvd2", hue="Ytrain",data=tsne_df,palette=sns.color_palette("hls", i),legend="full")
#     plt.show()
    

In [14]:
kmeans = KMeans(n_clusters=10, random_state=69).fit(X)
pred = kmeans.fit_predict(X)

In [15]:
cluster_centres = kmeans.cluster_centers_ 

In [16]:
print(cluster_centres)

[[-1.69559706e-01 -6.66120940e-01 -6.22062959e-01 -4.02248854e-01
  -6.74243673e-01  1.01103990e-01  6.26778071e-01 -1.88071546e-01
  -2.91237833e-02 -3.69646771e-01  8.70205176e-01 -4.58813455e-02
   1.29212882e+00  1.65835610e-01 -2.57706829e-01]
 [-3.08384677e-01 -9.93107428e-01 -3.02375600e-01 -1.34192230e+00
   7.04898799e-01  7.57052261e-02  1.60777506e-01  2.16137570e-02
   4.65456358e-02  2.71899384e-01 -4.73046250e-01  1.05737697e-01
  -4.09883398e-01  2.84747223e-01 -2.61987457e-01]
 [-1.21577241e+00  3.46679635e-02 -3.78774869e-01  8.09503552e-02
  -8.81645531e-01  8.44989425e-01 -6.75270186e-01  5.64863229e-02
   1.22903166e-02 -1.45498152e-01  3.17043512e-02  4.68575107e-02
  -5.20836934e-01 -1.07497279e-01  3.40590717e-01]
 [ 9.79621447e-01 -2.38532587e+00  2.58217426e+00  6.44844810e-01
  -4.88815294e-01 -5.33002609e-01 -1.62742638e-01 -1.08221183e-01
   3.37325044e-01  2.24539392e-01  1.17145945e-01 -1.10052827e-01
  -1.13557059e-01 -3.80544101e-01  1.34903486e-01]
 [ 2

In [17]:
print(len(X))
print(len(pred))

11159
11159


In [18]:
print(pred[0])

2


In [19]:
print(cluster_centres.shape)

(10, 15)


In [20]:
metrics_clusters = []
for i in range(10):
    s = 0
    c = 0 
    for j in range(len(X)):
        if pred[j] == i:
            s1 = 0
            for k in range(15):
                s1 += (X[j][k] - cluster_centres[i][k])**2
            s1 = s1**(0.5)
            s += s1
            c += 1
    s = s/c
    metrics_clusters.append(s)
            
        

In [21]:
metrics_clusters

[3.11099955061529,
 3.718583009312696,
 2.9656034782479557,
 2.9626330735784303,
 3.38198963631432,
 3.1678264853109876,
 4.134214203521528,
 2.823247858247276,
 3.40737732340161,
 4.68622137835526]

In [22]:
sum(metrics_clusters)/10

3.4358695996905353

In [23]:
y[0]

[165]

In [24]:
playlist_centres = []
playlist_counts = []
for i in range(169):
    a = []
    for j in range(15):
        a.append(0)
    playlist_centres.append(a)
    playlist_counts.append(0)

In [25]:
print(len(y))

11159


In [26]:
for i in range(len(X)):
    for c in y[i]:
        playlist_counts[c] += 1
        for j in range(15):
            playlist_centres[c][j] += X[i][j]
print(playlist_counts)
for i in range(169):
    for j in range(15):
        playlist_centres[i][j] = playlist_centres[i][j]/playlist_counts[i]
    

[65, 78, 74, 137, 91, 63, 56, 61, 59, 128, 71, 108, 99, 80, 98, 84, 109, 149, 89, 205, 74, 71, 118, 64, 100, 194, 58, 68, 67, 92, 852, 88, 106, 44, 117, 93, 87, 59, 250, 77, 147, 504, 714, 170, 309, 146, 451, 145, 204, 117, 144, 130, 127, 265, 166, 168, 129, 183, 153, 138, 310, 237, 239, 234, 146, 188, 164, 259, 489, 148, 62, 62, 203, 63, 92, 56, 242, 167, 171, 575, 309, 168, 162, 204, 215, 232, 82, 82, 83, 156, 78, 89, 104, 81, 112, 98, 73, 72, 56, 83, 86, 54, 80, 70, 274, 129, 124, 144, 110, 149, 109, 212, 311, 201, 168, 153, 139, 119, 415, 100, 87, 152, 108, 717, 202, 504, 265, 168, 155, 234, 160, 284, 689, 362, 106, 105, 73, 101, 76, 86, 78, 77, 141, 153, 69, 62, 66, 66, 78, 64, 190, 70, 111, 66, 131, 168, 63, 141, 168, 350, 168, 126, 127, 139, 275, 149, 122, 178, 150]


In [27]:
metrics_playlist = []
for i in range(169):
    s = 0
    c = 0 
    for j in range(len(X)):
        if i in y[j]:
            s1 = 0
            for k in range(15):
                s1 += (X[j][k] - playlist_centres[i][k])**2
            s1 = s1**(0.5)
            s += s1
            c += 1
    s = s/c
    metrics_playlist.append(s)
            
        

In [28]:
metrics_playlist

[3.3511491446668504,
 3.722642077005987,
 3.6255001662623387,
 3.7684085690947935,
 3.1641321510001084,
 3.8711963923745754,
 3.630206166022751,
 3.188970714503143,
 3.699774482991127,
 3.8866122271873174,
 3.01839603691762,
 2.9833352991899496,
 3.2346564522934145,
 3.0820769250147277,
 2.9932046317100443,
 3.4493399996707512,
 3.00992296512145,
 3.170953006362645,
 3.2948156139449103,
 3.6635192013715536,
 4.797057075843019,
 3.5139932842875523,
 3.6619270372438923,
 4.7321745493934655,
 3.651745416538459,
 3.5807467082507913,
 3.693303106820328,
 3.461266363720776,
 3.652477031384259,
 3.836529115662242,
 3.822182250311083,
 3.4973450261372436,
 3.9826725261343663,
 5.729128761176566,
 3.0585562887766327,
 4.368796605941351,
 3.0054399935203944,
 3.010934794081664,
 3.4024530723273987,
 3.7354034484200622,
 3.4132085864262445,
 3.7081600102052206,
 3.7179424742273404,
 3.373747545653792,
 3.629028188422454,
 2.5253601487446304,
 3.7306895028619222,
 3.6506008249599535,
 3.7821648666

In [29]:
sum(metrics_playlist)/169

3.468469618813997

In [30]:
# for n_clusters in range(2,11):
#     # Create a subplot with 1 row and 2 columns
#     fig, (ax1, ax2) = plt.subplots(1, 2)
#     fig.set_size_inches(18, 7)

#     # The 1st subplot is the silhouette plot
#     # The silhouette coefficient can range from -1, 1 but in this example all
#     # lie within [-0.1, 1]
#     ax1.set_xlim([-0.1, 1])
#     # The (n_clusters+1)*10 is for inserting blank space between silhouette
#     # plots of individual clusters, to demarcate them clearly.
#     ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

#     # Initialize the clusterer with n_clusters value and a random generator
#     # seed of 10 for reproducibility.
#     clusterer = KMeans(n_clusters=n_clusters, random_state=10)
#     cluster_labels = clusterer.fit_predict(X)

#     # The silhouette_score gives the average value for all the samples.
#     # This gives a perspective into the density and separation of the formed
#     # clusters
#     silhouette_avg = silhouette_score(X, cluster_labels)
#     print("For n_clusters =", n_clusters,
#           "The average silhouette_score is :", silhouette_avg)

#     # Compute the silhouette scores for each sample
#     sample_silhouette_values = silhouette_samples(X, cluster_labels)

#     y_lower = 10
#     for i in range(n_clusters):
#         # Aggregate the silhouette scores for samples belonging to
#         # cluster i, and sort them
#         ith_cluster_silhouette_values = \
#             sample_silhouette_values[cluster_labels == i]

#         ith_cluster_silhouette_values.sort()

#         size_cluster_i = ith_cluster_silhouette_values.shape[0]
#         y_upper = y_lower + size_cluster_i

#         color = cm.nipy_spectral(float(i) / n_clusters)
#         ax1.fill_betweenx(np.arange(y_lower, y_upper),
#                           0, ith_cluster_silhouette_values,
#                           facecolor=color, edgecolor=color, alpha=0.7)

#         # Label the silhouette plots with their cluster numbers at the middle
#         ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

#         # Compute the new y_lower for next plot
#         y_lower = y_upper + 10  # 10 for the 0 samples

#     ax1.set_title("The silhouette plot for the various clusters.")
#     ax1.set_xlabel("The silhouette coefficient values")
#     ax1.set_ylabel("Cluster label")

#     # The vertical line for average silhouette score of all the values
#     ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

#     ax1.set_yticks([])  # Clear the yaxis labels / ticks
#     ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

#     # 2nd Plot showing the actual clusters formed
#     colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
#     ax2.scatter(X[:, 0], X[:, 1], marker='.', s=30, lw=0, alpha=0.7,
#                 c=colors, edgecolor='k')

#     # Labeling the clusters
#     centers = clusterer.cluster_centers_
#     # Draw white circles at cluster centers
#     ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
#                 c="white", alpha=1, s=200, edgecolor='k')

#     for i, c in enumerate(centers):
#         ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
#                     s=50, edgecolor='k')

#     ax2.set_title("The visualization of the clustered data.")
#     ax2.set_xlabel("Feature space for the 1st feature")
#     ax2.set_ylabel("Feature space for the 2nd feature")

#     plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
#                   "with n_clusters = %d" % n_clusters),
#                  fontsize=14, fontweight='bold')

# plt.show()